In [1]:
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [58]:
censor_to_room_indx = {'r1' : 15, 'r2': 4, 'r3':24, 'r4': 30, 
                       'u1': 39, 'u2': 37, 'u3': 0, 'u4': 23,
                      'd1': [7,8], 'd2': [35,36], 'd3': [25,26], 'd4': [34,38]}

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.columns

Index(['Unnamed: 0', 'reliable_sensor1', 'reliable_sensor2',
       'reliable_sensor3', 'reliable_sensor4', 'unreliable_sensor1',
       'unreliable_sensor2', 'unreliable_sensor3', 'unreliable_sensor4',
       'robot1', 'robot2', 'door_sensor1', 'door_sensor2', 'door_sensor3',
       'door_sensor4', 'time', 'electricity_price', 'r1', 'r2', 'r3', 'r4',
       'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15',
       'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25',
       'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35',
       'c1', 'c2', 'c3', 'c4', 'o1', 'outside'],
      dtype='object')

In [4]:
data_copy = copy.deepcopy(data)
sensor_data = data[[ 'reliable_sensor1', 'reliable_sensor2',
       'reliable_sensor3', 'reliable_sensor4', 'unreliable_sensor1',
       'unreliable_sensor2', 'unreliable_sensor3', 'unreliable_sensor4',
       'robot1', 'robot2', 'door_sensor1', 'door_sensor2', 'door_sensor3',
       'door_sensor4', 'time']]

In [52]:
sensor_data.iloc[1].to_dict

<bound method Series.to_dict of reliable_sensor1       no motion
reliable_sensor2       no motion
reliable_sensor3          motion
reliable_sensor4       no motion
unreliable_sensor1     no motion
unreliable_sensor2     no motion
unreliable_sensor3     no motion
unreliable_sensor4     no motion
robot1                 ('r1', 0)
robot2                ('r19', 0)
door_sensor1                   0
door_sensor2                   0
door_sensor3                   0
door_sensor4                   0
time                    08:00:30
Name: 1, dtype: object>

In [72]:
tran_df =pd.read_csv("tran_matrix0.csv")
place_list = list(tran_df['place'])
print(place_list)
tran_matrix = tran_df.iloc[:,2:].values
tran_matrix

['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35', 'c1', 'c2', 'c3', 'c4', 'o1', 'outside']


array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.4688]])

In [57]:
tran_df

,Unnamed: 0,place,r1,r2,r3,r4,r5,r6,r7,r8,...,r32,r33,r34,r35,c1,c2,c3,c4,o1,outside
0,0,r1,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
1,1,r2,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
2,2,r3,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
3,3,r4,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
4,4,r5,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
5,5,r6,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
6,6,r7,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
7,7,r8,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
8,8,r9,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000
9,9,r10,0,0,0.0,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0.0,0,0.0000


In [18]:
initial_state = np.zeros(40)
initial_state = np.append(initial_state, 20)
print(initial_state)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. 20.]


In [69]:
initial_state[[7,8]] = [1,1]
initial_state[[7,8]] = [max(initial_state[7],2),max(initial_state[8],2)]


In [79]:
initial_state > 1

array([ True, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True])

In [20]:
new_matrix = initial_state @ tran_matrix
new_matrix

In [50]:
curr_state = initial_state
store_state =  initial_state[:35]
for i in range(4):
    curr_state = curr_state @ tran_matrix
    curr_state = censor_update(sensor_info,censor_to_room_indx, curr_state  )
    store_state = np.vstack((store_state,curr_state[:35]))

In [ ]:
def tran_calculation(curr_state,tran_matrix ):
    curr_state = 

In [55]:
'yes' == 0

False

In [76]:
robot_value = "('r27', 100)"
place_list.index('r27')

26

In [74]:
def robot_overwrite(robot_info, vector):
    "arguments: a string of robot information pair;"
    robot_info = robot_info.replace("(","")
    robot_info = robot_info.replace(")","")
    robot_info = robot_info.replace(" ","")
    robot_info = robot_info.replace("'","")
    robot_list = robot_info.split(",")
    vector[place_list.index(robot_list[0])] = int(robot_list[1]) 
    return(vector)


def censor_update(sensor_data, censor_room_match, state_vector):
    for censor_name, censor_status in sensor_data.items():
        if censor_status == None:
            continue 
        if censor_status == "no motion":
            continue
        if censor_status == 0:
            continue
        if censor_status == "motion":
            censor_abb = censor_name[0] + censor_name[-1]
            room = censor_room_match[censor_abb]
            state_vector[room] = max(state_vector[rooms], 1)
        if censor_status > 0:
            censor_abb = censor_name[0] + censor_name[-1]
            rooms = censor_room_match[censor_abb]
            state_vector[rooms] = [max(state_vector[rooms[0]], 1), max(state_vector[rooms[1]],1)]
        if censor_name[:5] == 'robot':
            state_vector = robot_overwrite(censor_status, state_vector)
    return(state_vector)

In [77]:
robot_overwrite(robot_value, initial_state)


array([  2.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   2.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0., 100.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,  20.])

In [91]:
thestring = "'lights1': 'off', 'lights2': 'on', 'lights3': 'off', 'lights4': 'off', 'lights5': 'off', 'lights6': 'off', 'lights7': 'off', 'lights8': 'off', 'lights9': 'off', 'lights10': 'off', 'lights11': 'off', 'lights12': 'off', 'lights13': 'off', 'lights14': 'off', 'lights15': 'off', 'lights16': 'off', 'lights17': 'off', 'lights18': 'off', 'lights19': 'off', 'lights20': 'off', 'lights21': 'off', 'lights22': 'off', 'lights23': 'off', 'lights24': 'off', 'lights25': 'off', 'lights26': 'off', 'lights27': 'off', 'lights28': 'off', 'lights29': 'off', 'lights30': 'off', 'lights31': 'off', 'lights32': 'off', 'lights33': 'off', 'lights34': 'off', 'lights35'"


In [92]:
new_str  = thestring.replace(": 'off'","")
new_str = new_str.replace(": 'on'","")
new_str= new_str.replace("'","")
new_str= new_str.replace(" ","")

In [94]:
print(new_str.split(",") )

['lights1', 'lights2', 'lights3', 'lights4', 'lights5', 'lights6', 'lights7', 'lights8', 'lights9', 'lights10', 'lights11', 'lights12', 'lights13', 'lights14', 'lights15', 'lights16', 'lights17', 'lights18', 'lights19', 'lights20', 'lights21', 'lights22', 'lights23', 'lights24', 'lights25', 'lights26', 'lights27', 'lights28', 'lights29', 'lights30', 'lights31', 'lights32', 'lights33', 'lights34', 'lights35']
